## Chest-Xray Pipeline Deployment in UCS

### Clone git repo

In [ ]:
! git clone https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

In [ ]:
! pip install kfp --user

### Import libraries

In [2]:
import kfp
import os
from kubernetes import client
import calendar
import time
import numpy as np

### Component files Declarations

In [3]:
component_root_train= 'components/v2/tf-model-train/'
component_root_serve = 'components/v2/tf-serving-deploy-service/'

#### Components Description

tf_train_model_op &nbsp;&nbsp;&nbsp; - &nbsp; Load dataset from nfs-volume, train Covid Xray model  and save model in nfs-volume. <br> 
tf_serve_model_op &nbsp; - &nbsp; Load Covid Xray model from nfs-volume and serve the model using tf-serving for later prediction. <br> 
tf_webui_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - &nbsp; Web application client UI loads Unlabeled data, does preprocessing, send API request to serving and display Predicted Location. <br> 
tf_tensor_op &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  - &nbsp; Tensorboard UI which provides the visualization metrics such as loss and accuracy, model graph (ops and layers) and histograms of weights, biases, or other tensors for Covid Xray Model <br> 

### Load components from respective .YAML config files

In [4]:
tf_train_model_op = kfp.components.load_component_from_file(os.path.join(component_root_train, 'component.yaml')) 
tf_serve_model_op = kfp.components.load_component_from_file(os.path.join(component_root_serve, 'component.yaml'))

### Define Volume and Volume Mounts

In [5]:
nfs_pvc = client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs')
nfs_volume = client.V1Volume(name='nfs', persistent_volume_claim=nfs_pvc)
nfs_volume_mount = client.V1VolumeMount(mount_path='/mnt/', name='nfs')

### Define pipeline function

In [6]:
# Creating timestamp
timestamp = str(calendar.timegm(time.gmtime()))
print(timestamp)
def covid_pipeline():
        
#     Defining Task for Model training
    tf_train_model_task = tf_train_model_op()
    tf_train_model_task.add_volume(nfs_volume)
    tf_train_model_task.add_volume_mount(nfs_volume_mount)
    
#     Defining Task to perform serving    
    tf_serve_model_task = tf_serve_model_op(timestamp=timestamp)
    tf_serve_model_task.add_volume(nfs_volume)
    tf_serve_model_task.add_volume_mount(nfs_volume_mount)
    tf_serve_model_task.after(tf_train_model_task)
    
#Creating a pipeline run
kfp.Client().create_run_from_pipeline_func(covid_pipeline, arguments={})

1591885185


RunPipelineResult(run_id=ce9b325c-6bbc-4a50-8782-a51982661161)

### TF Serving

In [8]:
## TF Serving
import tensorflow
sess = tensorflow.keras.backend.get_session()
img = tensorflow.read_file('image1.png')
img = tensorflow.image.decode_jpeg(img, channels=3)
img.set_shape([None, None, 3])
img = tensorflow.image.resize_images(img, (224, 224))
img = img.eval(session=sess) # convert to numpy array
img = np.expand_dims(img, 0) # make 'batch' of 1

In [12]:
import requests, json
headers = {"content-type": "application/json"}
data={"signature_name": "serving_default", "instances": img.tolist()}
json_response = requests.post('http://10.103.234.60:8500/v1/models/Model_Covid:predict', data=json.dumps(data), headers=headers)
prediction=json_response.json()['predictions']
labels=[1,0]
pred = labels[np.argmax(prediction)]
diction = {'0':'NORMAL','1':'COVID'}
result = diction[str(pred)]
print('Image result : ',result)

Image result :  NORMAL
